In [2]:
import pandas as pd

#### Train Dataset

In [ ]:
train = pd.read_csv('')

In [3]:
test = pd.read_csv('test_final_nonstd.csv', index_col='Unnamed: 0')

In [4]:
test = test[['cloudcover (%)',
 'relativehumidity_2m (%)',
 'elevation',
 'surface_pressure (hPa)',
 'cloudcover_mid (%)',
 'cloudcover_high (%)',
 'direct_normal_irradiance (W/m²)',
 'cloudcover_low (%)',
 'direct_radiation (W/m²)',
 'vapor_pressure_deficit (kPa)']]

In [5]:
test = pd.DataFrame(index=test.index, columns=test.columns, data = scaler.fit_transform(test))

NameError: name 'scaler' is not defined

In [ ]:
predictions = model.predict(test)

In [ ]:
result = pd.DataFrame({'rain_sum (mm)' : predictions})

In [ ]:
result.insert(0, 'id', result.index)

In [ ]:
result = result.set_index('id')

In [ ]:
def no_negatives(x) :
    if x < 0 :
        return 0
    return x

In [ ]:
result['rain_sum (mm)'] = result['rain_sum (mm)'].apply(lambda x : no_negatives(x))

In [ ]:
result.to_csv('percobaan_ke_sekian.csv')

In [6]:
train = pd.read_csv('merged_cesil.csv', index_col='Unnamed: 0')

In [7]:
train = train.drop('date', axis=1)

In [8]:
train['sunrise (hour)'] = pd.to_datetime(train['sunrise (iso8601)']).dt.hour
train['sunrise (minute)'] = pd.to_datetime(train['sunrise (iso8601)']).dt.minute
train['sunset (hour)'] = pd.to_datetime(train['sunset (iso8601)']).dt.hour
train['sunset (minute)'] = pd.to_datetime(train['sunset (iso8601)']).dt.minute

In [9]:
train_q = train[train['city'] == 10]
train_not_q = train[train['city'] != 10]

In [10]:
def nambah_waktu(time, x) :
    if x + time > 24 :
        return x - time
    return x + time

In [11]:
train_q['sunset (hour)'] = train_q['sunset (hour)'].apply(lambda x : nambah_waktu(12, x))

C:\Users\Rangga\AppData\Local\Temp\ipykernel_1748\3604849667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_q['sunset (hour)'] = train_q['sunset (hour)'].apply(lambda x : nambah_waktu(12, x))


In [12]:
train_use = pd.concat([train_not_q, train_q], axis=0)

In [13]:
train_use = train_use.drop(['sunrise (iso8601)', 'sunset (iso8601)'], axis=1)

In [14]:
train_use['sunrise_time'] = train_use['sunrise (hour)'].astype(str) + ':' + train_use['sunrise (minute)'].astype(str)
train_use['sunset_time'] = train_use['sunset (hour)'].astype(str) + ':' + train_use['sunset (minute)'].astype(str)

In [15]:
train_use['sunrise_time'] = pd.to_datetime(train_use['sunrise_time'], format='%H:%M')

In [16]:
train_use['sunset_time'] = pd.to_datetime(train_use['sunset_time'], format='%H:%M')

In [17]:
train_use['sunrise_to_sunset'] = (train_use['sunset_time'] - train_use['sunrise_time']).dt.total_seconds() / 3600

In [18]:
train_final = train_use.drop(['sunrise (hour)', 'sunrise (minute)', 'sunset (hour)', 'sunset (minute)', 'sunrise_time', 'sunset_time'], axis=1)

In [19]:
def min_to_pos(x) :
    if x < 0 :
        return abs(x)
    return x

In [20]:
train_final['sunrise_to_sunset'] = train_final['sunrise_to_sunset'].apply(lambda x: min_to_pos(x))

In [21]:
train_final_num = train_final.drop(['rain_sum (mm)', 'city'], axis=1)
train_final_cat = train_final[['rain_sum (mm)', 'city']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train_scaled = pd.DataFrame(index=train_final_num.index, columns=train_final_num.columns, data=scaler.fit_transform(train_final_num))

In [22]:
final = pd.concat([train_final_cat, train_scaled], axis=1)

In [26]:
final

,rain_sum (mm),city,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),shortwave_radiation_sum (MJ/m²),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),...,vapor_pressure_deficit (kPa),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),sunrise_to_sunset
0,7.7,1,0.869376,0.959612,0.877778,0.960110,0.542566,0.0,0.072027,0.109477,...,0.218928,0.893182,0.905715,0.929825,0.957341,0.644171,0.632397,0.585039,0.534251,0.814054
1,9.4,1,0.879536,0.954766,0.895062,0.954608,0.612893,0.0,0.075377,0.091503,...,0.244001,0.896143,0.906487,0.929898,0.956044,0.684645,0.660796,0.595571,0.534251,0.815135
2,7.8,1,0.896952,0.957997,0.927160,0.961486,0.630475,0.0,0.092127,0.117647,...,0.287327,0.900207,0.909787,0.930482,0.956044,0.646445,0.642984,0.606594,0.535541,0.814054
3,29.7,1,0.882438,0.959612,0.912346,0.960110,0.630475,0.0,0.077052,0.120915,...,0.229358,0.902479,0.912174,0.931798,0.956044,0.698768,0.673560,0.614370,0.536939,0.814054
4,22.5,1,0.883890,0.946688,0.893827,0.936726,0.619988,0.0,0.072027,0.117647,...,0.257039,0.899242,0.911050,0.932602,0.956044,0.706825,0.719573,0.659941,0.541456,0.814054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13253,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.017341,0.227204,0.234625,0.252997,0.268926,0.567014,0.406095,0.477165,0.461985,0.777297
13254,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.019548,0.222245,0.229570,0.249781,0.267399,0.567014,0.406095,0.477165,0.461985,0.777297
13255,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.018144,0.223278,0.228517,0.246564,0.266789,0.567014,0.406095,0.477165,0.461985,0.777297
13256,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.022356,0.224036,0.228517,0.244298,0.265568,0.567014,0.406095,0.477165,0.461985,0.777297


In [70]:
final = final.dropna(axis=0)

In [23]:
final.to_csv('normalized_anyg.csv')

In [25]:
final = pd.read_csv('20_train_final.csv', index_col='date')
final.drop(['Unnamed: 0', 'is_snowing.1'], axis=1, inplace=True)
# final.drop('is_snowing.1', axis=1, inplace=True)

In [1]:
import tensorflow as tf
import keras
import numpy as np

In [7]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten, GRU

In [26]:
X = final.drop('rain_sum (mm)', axis=1)
y = final['rain_sum (mm)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [27]:
X_train.shape

(10494, 21)

In [63]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences
X_train_series_sub = X_train.to_numpy().reshape((X_train.shape[0], subsequences, timesteps, 1))
X_valid_series_sub = X_test.to_numpy().reshape((X_test.shape[0], subsequences, timesteps, 1))

In [28]:
X_train = X_train.to_numpy().reshape((X_train.shape[0],  X_train.shape[1], 1))

In [29]:
X_test = X_test.to_numpy().reshape((X_test.shape[0],  X_test.shape[1], 1))

In [48]:
X_train.shape

(10494, 21, 1)

In [52]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))
regressor.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))
regressor.add(Dropout(0.2))
# Second GRU layer
regressor.add(GRU(units=64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='adam',loss='mean_squared_error', metrics=['acc'])
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=100,batch_size=100, validation_data=(X_test, y_test))

Epoch 1/100
105/105 [==============================] - 109s 891ms/step - loss: 74.6307 - acc: 0.1063 - val_loss: 42.8769 - val_acc: 0.1860
Epoch 2/100
105/105 [==============================] - 86s 810ms/step - loss: 56.3514 - acc: 0.1541 - val_loss: 41.6160 - val_acc: 0.1307
Epoch 3/100
105/105 [==============================] - 87s 832ms/step - loss: 57.1833 - acc: 0.1336 - val_loss: 42.2484 - val_acc: 0.0556
Epoch 4/100
105/105 [==============================] - 84s 796ms/step - loss: 54.3860 - acc: 0.0795 - val_loss: 50.5207 - val_acc: 0.1894
Epoch 5/100
105/105 [==============================] - 86s 814ms/step - loss: 54.5092 - acc: 0.0649 - val_loss: 39.8584 - val_acc: 0.1220
Epoch 6/100
105/105 [==============================] - 108s 1s/step - loss: 52.6314 - acc: 0.0842 - val_loss: 40.0471 - val_acc: 0.1753
Epoch 7/100
105/105 [==============================] - 76s 728ms/step - loss: 53.0164 - acc: 0.0843 - val_loss: 37.8334 - val_acc: 0.0652
Epoch 8/100
105/105 [==============

KeyboardInterrupt: 

In [64]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(Bidirectional(LSTM(270, activation='relu')))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer='adam', metrics=['acc'])

cnn_lstm_history = model_cnn_lstm.fit(X_train_series_sub, y_train, validation_data=(X_valid_series_sub, y_test), epochs=200, verbose=2)

Epoch 1/200
328/328 - 17s - loss: 65.5457 - acc: 0.0646 - val_loss: 42.7803 - val_acc: 0.0869
Epoch 2/200
328/328 - 14s - loss: 53.1960 - acc: 0.0912 - val_loss: 38.3327 - val_acc: 0.0911
Epoch 3/200
328/328 - 13s - loss: 49.4700 - acc: 0.0941 - val_loss: 37.0709 - val_acc: 0.0789
Epoch 4/200
328/328 - 13s - loss: 47.1043 - acc: 0.1041 - val_loss: 35.4245 - val_acc: 0.0960
Epoch 5/200
328/328 - 14s - loss: 45.5983 - acc: 0.1142 - val_loss: 42.3611 - val_acc: 0.1124
Epoch 6/200
328/328 - 13s - loss: 45.4171 - acc: 0.1214 - val_loss: 33.6217 - val_acc: 0.1208
Epoch 7/200
328/328 - 13s - loss: 43.6747 - acc: 0.1271 - val_loss: 35.3791 - val_acc: 0.1128
Epoch 8/200
328/328 - 12s - loss: 43.7179 - acc: 0.1263 - val_loss: 32.7544 - val_acc: 0.1288
Epoch 9/200
328/328 - 13s - loss: 43.1472 - acc: 0.1344 - val_loss: 33.5077 - val_acc: 0.1513
Epoch 10/200
328/328 - 13s - loss: 42.6661 - acc: 0.1357 - val_loss: 32.1467 - val_acc: 0.1505
Epoch 11/200
328/328 - 13s - loss: 41.7770 - acc: 0.1341 - 

In [47]:
# Build the model
print('Build model...')
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=False, input_shape=(22, 1))))
model.add(Dropout(0.4))
model.add(Dense(units=1))
model.add(Activation('relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])

batch_size=100
model.fit(X_train, y_train, batch_size=batch_size, epochs=200, validation_data=(X_test, y_test))

Build model...
Epoch 1/200
105/105 [==============================] - 26s 118ms/step - loss: 99.4284 - acc: 0.0408 - val_loss: 76.6058 - val_acc: 0.0111
Epoch 2/200
105/105 [==============================] - 9s 85ms/step - loss: 90.8416 - acc: 0.0095 - val_loss: 73.3644 - val_acc: 0.0111
Epoch 3/200
105/105 [==============================] - 9s 87ms/step - loss: 75.3904 - acc: 0.0307 - val_loss: 43.5846 - val_acc: 0.1532
Epoch 4/200
105/105 [==============================] - 15s 142ms/step - loss: 54.5992 - acc: 0.1297 - val_loss: 39.0204 - val_acc: 0.1479
Epoch 5/200
105/105 [==============================] - 11s 101ms/step - loss: 52.4774 - acc: 0.1384 - val_loss: 39.9407 - val_acc: 0.1159
Epoch 6/200
105/105 [==============================] - 9s 90ms/step - loss: 51.1887 - acc: 0.1394 - val_loss: 36.9649 - val_acc: 0.1566
Epoch 7/200
105/105 [==============================] - 10s 99ms/step - loss: 50.3240 - acc: 0.1414 - val_loss: 36.5510 - val_acc: 0.1509
Epoch 8/200
105/105 [=====

In [48]:
# Define the model
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]))))
model.add(keras.layers.Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model to the training data
model.fit(X_train, y_train, epochs=200, batch_size=100)

# Use the model to make predictions on new data
y_pred = model.predict(X_test)

Epoch 1/100
328/328 [==============================] - 15s 24ms/step - loss: 71.2355
Epoch 2/100
328/328 [==============================] - 7s 20ms/step - loss: 54.5865
Epoch 3/100
328/328 [==============================] - 7s 22ms/step - loss: 51.0685
Epoch 4/100
328/328 [==============================] - 7s 21ms/step - loss: 49.5541
Epoch 5/100
328/328 [==============================] - 7s 21ms/step - loss: 48.9862
Epoch 6/100
328/328 [==============================] - 7s 23ms/step - loss: 49.9401
Epoch 7/100
328/328 [==============================] - 7s 21ms/step - loss: 50.0161
Epoch 8/100
328/328 [==============================] - 7s 20ms/step - loss: 47.3786
Epoch 9/100
328/328 [==============================] - 7s 21ms/step - loss: 48.3201
Epoch 10/100
328/328 [==============================] - 7s 22ms/step - loss: 48.0859
Epoch 11/100
328/328 [==============================] - 8s 24ms/step - loss: 48.8775
Epoch 12/100
328/328 [==============================] - 7s 20ms/step - lo

In [54]:
test = pd.read_csv('20_test_final.csv', index_col='Unnamed: 0')
# test.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
# test['is_snowing.1'] = test['is_snowing']

In [33]:
test

,cloudcover (%),relativehumidity_2m (%),elevation,surface_pressure (hPa),cloudcover_mid (%),cloudcover_high (%),direct_normal_irradiance (W/m²),cloudcover_low (%),direct_radiation (W/m²),vapor_pressure_deficit (kPa),...,windspeed_100m (km/h),soil_moisture_0_to_7cm (m³/m³),shortwave_radiation (W/m²),shortwave_radiation_sum (MJ/m²),diffuse_radiation (W/m²),soil_moisture_7_to_28cm (m³/m³),sunrise_to_sunset,et0_fao_evapotranspiration (mm)_y,is_snowing,is_snowing.1
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.136250,0.450593,0.0,0.952183,0.093072,0.026667,0.373827,0.081563,0.617488,0.716211,...,0.089102,0.085616,0.642953,0.642857,0.235847,0.073812,0.90537,0.706349,0,0
2018-01-02,0.205000,0.511199,0.0,0.955921,0.192821,0.124583,0.343263,0.059473,0.569625,0.639158,...,0.119778,0.095829,0.618682,0.618738,0.267760,0.070229,0.90537,0.685714,0,0
2018-01-03,0.166667,0.528327,0.0,0.956826,0.065526,0.234167,0.380366,0.063721,0.621170,0.621053,...,0.205205,0.092584,0.646181,0.645949,0.236066,0.066544,0.90537,0.750794,0,0
2018-01-04,0.217500,0.534914,0.0,0.957979,0.051753,0.533333,0.408646,0.029737,0.676660,0.616211,...,0.199805,0.090198,0.686150,0.686147,0.222295,0.063984,0.90537,0.787302,0,0
2018-01-05,0.287083,0.552701,0.0,0.956578,0.022538,0.848333,0.390094,0.021665,0.647469,0.591368,...,0.260407,0.088480,0.675351,0.675325,0.249617,0.061323,0.90581,0.792063,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-03,0.951667,0.287879,0.0,0.873003,0.996244,0.942917,0.000000,0.116398,0.000000,0.047158,...,0.433811,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.076190,1,1
2020-10-04,1.000000,0.404480,0.0,0.859104,0.970785,0.952917,0.000000,0.525489,0.000000,0.064632,...,0.407260,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.080952,1,1
2020-10-05,0.739167,0.615283,0.0,0.887137,0.173623,0.035417,0.000000,0.788020,0.000000,0.024421,...,0.152254,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.004762,1,1


In [47]:
cek = pd.read_csv('20_test_final.csv', index_col='Unnamed: 0')

In [20]:
test

,cloudcover (%),relativehumidity_2m (%),elevation,surface_pressure (hPa),cloudcover_mid (%),cloudcover_high (%),direct_normal_irradiance (W/m²),cloudcover_low (%),direct_radiation (W/m²),vapor_pressure_deficit (kPa),...,windspeed_10m_max (km/h),windspeed_100m (km/h),soil_moisture_0_to_7cm (m³/m³),shortwave_radiation (W/m²),shortwave_radiation_sum (MJ/m²),diffuse_radiation (W/m²),soil_moisture_7_to_28cm (m³/m³),sunrise_to_sunset,et0_fao_evapotranspiration (mm)_y,is_snowing
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.136250,0.450593,0.0,0.952183,0.093072,0.026667,0.373827,0.081563,0.617488,0.716211,...,0.097701,0.089102,0.085616,0.642953,0.642857,0.235847,0.073812,0.90537,0.706349,0
2018-01-02,0.205000,0.511199,0.0,0.955921,0.192821,0.124583,0.343263,0.059473,0.569625,0.639158,...,0.147510,0.119778,0.095829,0.618682,0.618738,0.267760,0.070229,0.90537,0.685714,0
2018-01-03,0.166667,0.528327,0.0,0.956826,0.065526,0.234167,0.380366,0.063721,0.621170,0.621053,...,0.283525,0.205205,0.092584,0.646181,0.645949,0.236066,0.066544,0.90537,0.750794,0
2018-01-04,0.217500,0.534914,0.0,0.957979,0.051753,0.533333,0.408646,0.029737,0.676660,0.616211,...,0.254789,0.199805,0.090198,0.686150,0.686147,0.222295,0.063984,0.90537,0.787302,0
2018-01-05,0.287083,0.552701,0.0,0.956578,0.022538,0.848333,0.390094,0.021665,0.647469,0.591368,...,0.339080,0.260407,0.088480,0.675351,0.675325,0.249617,0.061323,0.90581,0.792063,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-03,0.951667,0.287879,0.0,0.873003,0.996244,0.942917,0.000000,0.116398,0.000000,0.047158,...,0.367816,0.433811,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.076190,1
2020-10-04,1.000000,0.404480,0.0,0.859104,0.970785,0.952917,0.000000,0.525489,0.000000,0.064632,...,0.365900,0.407260,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.080952,1
2020-10-05,0.739167,0.615283,0.0,0.887137,0.173623,0.035417,0.000000,0.788020,0.000000,0.024421,...,0.222222,0.152254,0.560943,0.000000,0.000000,0.000000,0.387080,0.59243,0.004762,1


In [36]:
test.shape

(4972, 21)

In [34]:
X_train.shape

(10494, 21, 1)

In [37]:
# subsequences = 2
# timesteps = test.shape[1]//subsequences
test = test.to_numpy().reshape((test.shape[0], test.shape[1], 1))

In [38]:
predictions = regressor.predict(test)

In [39]:
predictions.shape

(4972, 1)

In [40]:
predictions2 = predictions.reshape((predictions.shape[0],))

In [41]:
result = pd.DataFrame({'rain_sum (mm)' : predictions2})

In [58]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4972 entries, 0 to 4971
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rain_sum (mm)  4972 non-null   float64
dtypes: float64(1)
memory usage: 77.7 KB


In [57]:
result

,rain_sum (mm)
id,
0,0.346421
1,0.309038
2,0.174983
3,0.534120
4,0.467689
...,...
4967,0.000000
4968,0.413800
4969,0.000000


In [43]:
result['rain_sum (mm)'] = result['rain_sum (mm)'].apply(lambda x: 0 if x < 0 else x)

In [44]:
result[result['rain_sum (mm)'] < 0]

,rain_sum (mm)


In [45]:
result.insert(0, 'id', result.index)
result = result.set_index('id')

In [81]:
result.to_csv('MKIN_GEDE_MKIN_BAGUS_2.csv')

In [46]:
result['rain_sum (mm)'].max()

57.1710090637207

In [47]:
result['rain_sum (mm)'].mean()

7.437733345399052

In [79]:
cek_snow = pd.concat([cek, result], axis=1)

In [80]:
cek_snow.groupby('is_snowing').mean()

,cloudcover (%),relativehumidity_2m (%),elevation,surface_pressure (hPa),cloudcover_mid (%),cloudcover_high (%),direct_normal_irradiance (W/m²),cloudcover_low (%),direct_radiation (W/m²),vapor_pressure_deficit (kPa),...,windspeed_10m_max (km/h),windspeed_100m (km/h),soil_moisture_0_to_7cm (m³/m³),shortwave_radiation (W/m²),shortwave_radiation_sum (MJ/m²),diffuse_radiation (W/m²),soil_moisture_7_to_28cm (m³/m³),sunrise_to_sunset,et0_fao_evapotranspiration (mm)_y,rain_sum (mm)
is_snowing,,,,,,,,,,,,,,,,,,,,,
0,0.591332,0.754006,0.157691,0.765771,0.328709,0.694971,0.234595,0.264861,0.390155,0.213627,...,0.157974,0.131600,0.661866,0.522059,0.521959,0.376395,0.646745,0.881295,0.526059,5.938230
1,0.774165,0.576431,0.002934,0.867834,0.659965,0.282457,0.123388,0.487280,0.111703,0.052884,...,0.331403,0.292899,0.582181,0.202565,0.202536,0.212007,0.577159,0.631235,0.109842,0.458942


In [147]:
cek_snow.groupby('is_snowing').mean()

,Unnamed: 0,cloudcover (%),relativehumidity_2m (%),elevation,surface_pressure (hPa),cloudcover_mid (%),cloudcover_high (%),direct_normal_irradiance (W/m²),cloudcover_low (%),direct_radiation (W/m²),vapor_pressure_deficit (kPa),sunrise_to_sunset,rain_sum (mm)
is_snowing,,,,,,,,,,,,,
0,2373.003186,0.591332,0.754006,0.157691,0.765771,0.328709,0.694971,0.234595,0.264861,0.390155,0.213627,0.881295,6323846.0
1,4491.693182,0.774165,0.576431,0.002934,0.867834,0.659965,0.282457,0.123388,0.487280,0.111703,0.052884,0.631235,11973230.0


In [119]:
result[result['rain_sum (mm)'] == 0]

,rain_sum (mm)


In [ ]:
pke = final[['rain_sum (mm)',
 'cloudcover (%)',
 'relativehumidity_2m (%)',
 'elevation',
 'surface_pressure (hPa)',
 'cloudcover_mid (%)',
 'cloudcover_high (%)',
 'direct_normal_irradiance (W/m²)',
 'cloudcover_low (%)',
 'direct_radiation (W/m²)',
 'vapor_pressure_deficit (kPa)'
]]

In [ ]:
pke = pke.dropna(axis=0)

In [ ]:
X = pke.drop('rain_sum (mm)', axis=1)
y = pke['rain_sum (mm)']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=70)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
train_cat = train[['city', 'rain_sum (mm)']]

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
snowing_cities = train[train['snowfall_sum (cm)'] > 0]
snowing_cities['city'].unique() # bisa buat features engineering kota bersalju

array(['sa', 'q'], dtype=object)

In [ ]:
# kota t kota hujan
no_rain = (train[train['rain_sum (mm)'] == 0])

In [ ]:
cities = list(no_rain['city'].unique())

In [ ]:
no_rain[pd.to_datetime(no_rain['time']).dt.year == 2022]

,time,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),sunrise (iso8601),sunset (iso8601),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),et0_fao_evapotranspiration (mm),elevation,city
11932,2022-02-11,-1.1,-8.3,-4.5,-12.1,2022-02-11T04:36,2022-02-11T15:01,12.26,0.0,0.00,3.9,10.1,NaN,0.90,17,sa
11934,2022-10-25,10.6,3.8,7.3,0.3,2022-10-25T03:58,2022-10-25T14:38,13.21,0.0,0.00,9.8,22.7,NaN,1.78,17,sa
13251,2022-04-23,-11.4,-14.6,-15.7,-18.5,1970-01-01T07:00,1970-01-01T07:00,12.86,0.0,1.05,6.1,9.7,NaN,0.59,0,q


In [ ]:
cold_cities = train.loc[(train.city == 'sa') | (train.city == 'q')]

In [ ]:
cold_cities[pd.to_datetime(cold_cities['time']).dt.year == 2021]

,time,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),sunrise (iso8601),sunset (iso8601),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),et0_fao_evapotranspiration (mm),elevation,city
11719,2021-01-01,-8.9,-12.2,-15.2,-18.5,2021-01-01T05:04,2021-01-01T14:12,6.80,0.0,2.52,27.6,46.1,277.0,0.57,17,sa
11720,2021-01-02,-6.3,-13.3,-11.6,-19.5,2021-01-02T05:04,2021-01-02T14:12,6.82,0.0,2.31,18.4,31.7,266.0,0.48,17,sa
11721,2021-01-03,-4.0,-10.0,-10.0,-15.7,2021-01-03T05:04,2021-01-03T14:13,4.54,0.0,1.82,20.5,34.6,258.0,0.63,17,sa
11722,2021-01-04,-4.5,-9.5,-10.8,-15.7,2021-01-04T05:04,2021-01-04T14:14,6.65,0.0,2.31,23.3,42.5,279.0,0.69,17,sa
11723,2021-01-05,-5.0,-10.7,-10.3,-16.8,2021-01-05T05:04,2021-01-05T14:15,7.72,0.0,1.82,20.6,35.3,276.0,0.55,17,sa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13246,2021-07-29,9.5,6.9,5.2,3.0,1970-01-01T07:00,1970-01-01T07:00,8.00,0.4,0.00,21.8,32.8,67.0,1.66,0,q
13247,2021-07-30,6.8,5.0,3.0,1.1,1970-01-01T07:00,1970-01-01T07:00,8.14,2.8,0.00,16.1,25.6,54.0,1.26,0,q
13248,2021-07-31,6.0,3.7,2.9,0.7,1970-01-01T07:00,1970-01-01T07:00,9.30,2.1,0.00,16.1,21.2,352.0,1.20,0,q
13249,2021-08-01,4.3,3.7,1.5,0.7,1970-01-01T07:00,1970-01-01T07:00,15.36,0.0,0.00,9.7,15.5,335.0,1.49,0,q


In [ ]:
cold_cities[pd.to_datetime(cold_cities['time']).dt.year == 2022]

,time,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),sunrise (iso8601),sunset (iso8601),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),et0_fao_evapotranspiration (mm),elevation,city
11932,2022-02-11,-1.1,-8.3,-4.5,-12.1,2022-02-11T04:36,2022-02-11T15:01,12.26,0.0,0.00,3.9,10.1,NaN,0.90,17,sa
11933,2022-06-30,19.9,16.9,22.0,16.6,2022-06-30T01:56,2022-06-30T17:19,12.14,1.1,0.00,13.3,24.5,NaN,2.18,17,sa
11934,2022-10-25,10.6,3.8,7.3,0.3,2022-10-25T03:58,2022-10-25T14:38,13.21,0.0,0.00,9.8,22.7,NaN,1.78,17,sa
11935,2022-12-23,0.9,-0.1,-2.6,-3.3,2022-12-23T05:01,2022-12-23T14:05,NaN,NaN,NaN,12.1,15.5,NaN,NaN,17,sa
11936,2022-12-24,NaN,NaN,NaN,NaN,2022-12-24T05:02,2022-12-24T14:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,sa
11937,2022-12-25,NaN,NaN,NaN,NaN,2022-12-25T05:02,2022-12-25T14:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,sa
11938,2022-12-26,NaN,NaN,NaN,NaN,2022-12-26T05:03,2022-12-26T14:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,sa
11939,2022-12-27,NaN,NaN,NaN,NaN,2022-12-27T05:03,2022-12-27T14:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,sa
11940,2022-12-28,NaN,NaN,NaN,NaN,2022-12-28T05:03,2022-12-28T14:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,sa
13251,2022-04-23,-11.4,-14.6,-15.7,-18.5,1970-01-01T07:00,1970-01-01T07:00,12.86,0.0,1.05,6.1,9.7,NaN,0.59,0,q


In [ ]:
for year in range(2018, 2023) :
    print(year)
    for city in cities :
        print(f'\t+{city}')
        current = no_rain.loc[(pd.to_datetime(no_rain['time']).dt.year == year) & (no_rain['city'] == city)]
        current_months = list(pd.to_datetime(current['time']).dt.month.unique())
        for month in current_months :
            per_month = len(current.loc[pd.to_datetime(current['time']).dt.month == month])
            print(f'\t\t-bulan {month} : {per_month} kali ga hujan')

2018
	+su
		-bulan 4 : 2 kali ga hujan
		-bulan 5 : 17 kali ga hujan
		-bulan 6 : 14 kali ga hujan
		-bulan 7 : 29 kali ga hujan
		-bulan 8 : 28 kali ga hujan
		-bulan 9 : 23 kali ga hujan
		-bulan 10 : 12 kali ga hujan
		-bulan 11 : 1 kali ga hujan
		-bulan 12 : 1 kali ga hujan
	+si
		-bulan 3 : 1 kali ga hujan
		-bulan 4 : 1 kali ga hujan
		-bulan 5 : 14 kali ga hujan
		-bulan 6 : 7 kali ga hujan
		-bulan 7 : 17 kali ga hujan
		-bulan 8 : 19 kali ga hujan
		-bulan 9 : 12 kali ga hujan
		-bulan 10 : 13 kali ga hujan
		-bulan 11 : 1 kali ga hujan
	+u
		-bulan 2 : 1 kali ga hujan
		-bulan 3 : 3 kali ga hujan
		-bulan 4 : 5 kali ga hujan
		-bulan 5 : 17 kali ga hujan
		-bulan 6 : 12 kali ga hujan
		-bulan 7 : 13 kali ga hujan
		-bulan 8 : 14 kali ga hujan
		-bulan 9 : 15 kali ga hujan
		-bulan 10 : 10 kali ga hujan
	+le
		-bulan 3 : 2 kali ga hujan
		-bulan 4 : 3 kali ga hujan
		-bulan 5 : 11 kali ga hujan
		-bulan 6 : 10 kali ga hujan
		-bulan 7 : 28 kali ga hujan
		-bulan 8 : 21 kali g

In [ ]:
# referensi nambah kolom rain probability (in %)
for i in range(len(no_rain_months)) :
    current = no_rain.loc[pd.to_datetime(no_rain['time']).dt.month == no_rain_months[i]]
    length = len(current)
    print(f"month {no_rain_months[i]} : {length}")
    for num, city in enumerate(cities) :
        length_city = len(current[current['city'] == city])
        print(f"{num + 1}. {city} : {length_city}")

month 4 : 246
1. su : 11
2. si : 9
3. u : 23
4. le : 5
5. p : 6
6. lh : 14
7. b : 0
8. t : 0
9. sa : 58
10. q : 120
month 5 : 352
1. su : 49
2. si : 36
3. u : 58
4. le : 21
5. p : 3
6. lh : 5
7. b : 1
8. t : 0
9. sa : 56
10. q : 123
month 6 : 409
1. su : 65
2. si : 47
3. u : 59
4. le : 45
5. p : 6
6. lh : 20
7. b : 8
8. t : 0
9. sa : 51
10. q : 108
month 7 : 507
1. su : 107
2. si : 70
3. u : 56
4. le : 77
5. p : 11
6. lh : 28
7. b : 5
8. t : 0
9. sa : 67
10. q : 86
month 8 : 395
1. su : 76
2. si : 55
3. u : 43
4. le : 67
5. p : 16
6. lh : 23
7. b : 11
8. t : 0
9. sa : 36
10. q : 68
month 9 : 365
1. su : 72
2. si : 37
3. u : 41
4. le : 60
5. p : 26
6. lh : 8
7. b : 1
8. t : 0
9. sa : 31
10. q : 89
month 10 : 236
1. su : 43
2. si : 25
3. u : 22
4. le : 25
5. p : 0
6. lh : 0
7. b : 2
8. t : 0
9. sa : 31
10. q : 88
month 11 : 144
1. su : 4
2. si : 10
3. u : 11
4. le : 6
5. p : 2
6. lh : 0
7. b : 0
8. t : 0
9. sa : 21
10. q : 90
month 12 : 169
1. su : 3
2. si : 2
3. u : 3
4. le : 2
5. p : 2

In [ ]:
len(no_rain) / len(train) * 100

27.198672499622866

In [ ]:
for dates in no_rain['time'].unique() :
    print(dates)

2018-04-10
2018-04-29
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-14
2018-05-22
2018-05-28
2018-05-29
2018-05-30
2018-06-03
2018-06-06
2018-06-10
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-25
2018-06-26
2018-06-28
2018-06-29
2018-06-30
2018-07-01
2018-07-02
2018-07-03
2018-07-04
2018-07-05
2018-07-06
2018-07-07
2018-07-08
2018-07-09
2018-07-10
2018-07-11
2018-07-12
2018-07-13
2018-07-14
2018-07-15
2018-07-16
2018-07-17
2018-07-18
2018-07-19
2018-07-20
2018-07-21
2018-07-23
2018-07-24
2018-07-25
2018-07-26
2018-07-27
2018-07-28
2018-07-29
2018-07-31
2018-08-02
2018-08-03
2018-08-04
2018-08-05
2018-08-06
2018-08-07
2018-08-08
2018-08-09
2018-08-10
2018-08-11
2018-08-12
2018-08-13
2018-08-14
2018-08-15
2018-08-16
2018-08-17
2018-08-18
2018-08-19
2018-08-20
2018-08-21
2018-08-22
2018-08-23
2018-08-24
2018-08-25
2018-08-26
2018-08-27
2018-08-28
2018-08-30
2018-09-03

In [ ]:
train = pd.read_csv('train_merged_ala_rangga_selisih_sunset_sunrise.csv')
train.drop(['Unnamed: 0', 'sunrise (iso8601)', 'sunset (iso8601)'], axis=1, inplace=True)
train

,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),...,soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),year,month,day,sunrise to sunset (hours)
0,29.900000,26.000000,36.300000,31.600000,17.590000,7.70000,0.000000,6.900000,20.200000,277.000000,...,28.700000,29.270833,0.297917,0.300292,0.282667,0.280000,2018,1,1,12.566667
1,30.600000,25.700000,37.700000,31.200000,19.870000,9.40000,0.000000,7.100000,18.000000,168.000000,...,28.704167,29.200000,0.315708,0.312250,0.287125,0.280000,2018,1,2,12.583333
2,31.800000,25.900000,40.300000,31.700000,20.440000,7.80000,0.000000,8.100000,21.200000,125.000000,...,28.737500,29.200000,0.298917,0.304750,0.291792,0.280500,2018,1,3,12.566667
3,30.800000,26.000000,39.100000,31.600000,20.440000,29.70000,0.000000,7.200000,21.600000,190.000000,...,28.812500,29.200000,0.321917,0.317625,0.295083,0.281042,2018,1,4,12.566667
4,30.900000,25.200000,37.600000,29.900000,20.100000,22.50000,0.000000,6.900000,21.200000,241.000000,...,28.858333,29.200000,0.325458,0.337000,0.314375,0.282792,2018,1,5,12.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13253,-7.078963,-10.350686,-11.474619,-15.087271,9.284882,0.13341,0.258375,12.583841,20.128354,85.983908,...,-9.879167,-8.316667,0.264000,0.205000,0.237000,0.252000,2022,12,24,0.000000
13254,-7.078963,-10.350686,-11.474619,-15.087271,9.284882,0.13341,0.258375,12.583841,20.128354,85.983908,...,-10.062500,-8.400000,0.264000,0.205000,0.237000,0.252000,2022,12,25,0.000000
13255,-7.078963,-10.350686,-11.474619,-15.087271,9.284882,0.13341,0.258375,12.583841,20.128354,85.983908,...,-10.245833,-8.433333,0.264000,0.205000,0.237000,0.252000,2022,12,26,0.000000
13256,-7.078963,-10.350686,-11.474619,-15.087271,9.284882,0.13341,0.258375,12.583841,20.128354,85.983908,...,-10.375000,-8.500000,0.264000,0.205000,0.237000,0.252000,2022,12,27,0.000000


In [ ]:
train[train['sunrise to sunset (hours)'] < 0]

,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),...,soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),year,month,day,sunrise to sunset (hours)
11984,-19.7,-24.4,-24.0,-29.9,0.00,0.0,0.00,12.0,22.3,32.0,...,-15.370833,-12.500000,0.334,0.324,0.267,0.250,2018,2,13,-22.433333
11985,-14.6,-21.7,-20.3,-26.9,0.00,0.0,0.00,17.7,28.8,357.0,...,-15.479167,-12.595833,0.334,0.324,0.267,0.250,2018,2,14,-21.616667
11986,-14.4,-17.4,-19.7,-23.2,0.00,0.0,0.00,19.5,31.0,30.0,...,-15.562500,-12.691667,0.334,0.324,0.267,0.250,2018,2,15,-20.983333
11987,-9.4,-16.1,-14.8,-22.1,0.00,0.0,0.00,19.1,30.6,31.0,...,-15.600000,-12.787500,0.334,0.324,0.267,0.250,2018,2,16,-20.450000
11988,-12.1,-16.3,-16.1,-21.7,0.01,0.0,0.00,13.3,28.8,253.0,...,-15.616667,-12.879167,0.334,0.324,0.267,0.250,2018,2,17,-20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13141,-15.4,-18.9,-20.0,-23.3,13.68,0.0,0.00,8.4,16.9,284.0,...,-18.875000,-18.500000,0.267,0.280,0.246,0.253,2021,4,15,-3.866667
13142,-11.6,-16.3,-17.4,-21.6,14.90,0.0,0.00,21.0,37.1,354.0,...,-18.862500,-18.500000,0.267,0.280,0.246,0.253,2021,4,16,-3.383333
13143,-10.9,-14.8,-15.1,-19.6,13.27,0.0,0.00,10.2,26.6,263.0,...,-18.900000,-18.504167,0.267,0.280,0.246,0.253,2021,4,17,-2.833333
13144,-12.9,-16.2,-17.2,-21.4,12.16,0.0,0.07,9.9,14.8,85.0,...,-18.900000,-18.595833,0.267,0.280,0.246,0.253,2021,4,18,-2.150000


In [ ]:
city_dic = {'su' : '1', 'si' : '2', 'u' : '3', 'le' : '4', 'p' : '5', 'lh' : '6', 'b' : '7', 't' : '8', 'sa' : '9', 'q' : '10'}

In [ ]:
train['city'] = train['city'].map(city_dic)

In [ ]:
train_cat = train[['city', 'year', 'month', 'day']]
train_num = train.drop(['city', 'year', 'month', 'day'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_num_scaled = pd.DataFrame(columns=train_num.columns, index=train_num.index, data=scaler.fit_transform(train_num))

In [ ]:
train_final = pd.concat([train_cat, train_num_scaled], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X = train_final.drop('rain_sum (mm)', axis=1)
y = train_final['rain_sum (mm)']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=70)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
prediction = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, prediction)
rmse

0.3709810911793752

#### Test Data

In [ ]:
test = pd.read_csv('test.csv')
test_hourly = pd.read_csv('test_hourly.csv')

In [ ]:
test_hourly['time'] = pd.to_datetime(test_hourly['time'])

In [ ]:
test_hourly['date'] = test_hourly['time'].dt.date

In [ ]:
test_hourly.drop('time', axis=1, inplace=True)

In [ ]:
test_hourly_mean = test_hourly.groupby(['date', 'city']).mean()

In [ ]:
test_hourly_mean = test_hourly_mean.reset_index()

In [ ]:
test_hourly_mean['date'] = test_hourly_mean['date'].astype(str)

In [ ]:
test.drop('id', axis=1, inplace=True)

In [ ]:
test.rename(columns={'time' : 'date'}, inplace=True)
test_merged = pd.merge(test, test_hourly_mean, how='inner', on=['date', 'city'])

In [ ]:
test_merged['sunrise (iso8601)'] = pd.to_datetime(test_merged['sunrise (iso8601)'])
test_merged['sunset (iso8601)'] = pd.to_datetime(test_merged['sunset (iso8601)'])
test_merged

,date,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),sunrise (iso8601),sunset (iso8601),shortwave_radiation_sum (MJ/m²),snowfall_sum (cm),windspeed_10m_max (km/h),...,et0_fao_evapotranspiration (mm)_y,vapor_pressure_deficit (kPa),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³)
0,2021-08-02,32.3,25.3,38.2,29.6,2021-08-02 05:39:00,2021-08-02 17:30:00,20.79,0.00,7.6,...,0.185417,1.423333,31.445833,30.625000,29.233333,28.600000,0.056500,0.077500,0.112500,0.234000
1,2021-08-03,31.7,25.5,37.4,30.3,2021-08-03 05:39:00,2021-08-03 17:30:00,20.01,0.00,10.2,...,0.180000,1.270833,30.937500,30.591667,29.333333,28.600000,0.060958,0.076042,0.111583,0.233292
2,2021-08-04,32.6,25.7,37.1,30.8,2021-08-04 05:39:00,2021-08-04 17:30:00,20.89,0.00,17.3,...,0.197083,1.235000,31.033333,30.558333,29.416667,28.687500,0.059542,0.074542,0.110917,0.233000
3,2021-08-05,32.0,25.7,36.8,31.1,2021-08-05 05:39:00,2021-08-05 17:30:00,22.19,0.00,15.8,...,0.206667,1.225417,30.837500,30.487500,29.500000,28.700000,0.058500,0.073500,0.110375,0.232458
4,2021-08-06,31.9,26.3,36.0,31.6,2021-08-06 05:38:00,2021-08-06 17:30:00,21.84,0.00,20.2,...,0.207917,1.176250,30.654167,30.400000,29.516667,28.700000,0.057750,0.072417,0.109458,0.232000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4967,2022-12-18,-6.9,-13.1,-13.5,-19.5,1970-01-01 07:00:00,1970-01-01 07:00:00,0.00,1.12,21.7,...,0.020000,0.099167,-10.487500,-10.275000,-9.575000,-8.058333,0.264000,0.205000,0.237000,0.252000
4968,2022-12-19,-2.5,-6.8,-7.1,-13.4,1970-01-01 07:00:00,1970-01-01 07:00:00,0.00,1.40,21.6,...,0.021250,0.133750,-10.308333,-10.183333,-9.600000,-8.100000,0.264000,0.205000,0.237000,0.252000
4969,2022-12-20,-4.6,-16.9,-9.0,-21.9,1970-01-01 07:00:00,1970-01-01 07:00:00,0.00,1.05,14.1,...,0.001250,0.054167,-10.137500,-10.025000,-9.600000,-8.154167,0.264000,0.205000,0.237000,0.252000
4970,2022-12-21,-17.2,-21.2,-22.2,-26.5,1970-01-01 07:00:00,1970-01-01 07:00:00,0.00,0.00,10.8,...,0.000000,0.029167,-10.462500,-10.183333,-9.600000,-8.200000,0.264000,0.205000,0.237000,0.252000


In [ ]:
test_merged['sunrise to sunset (hours)'] = (test_merged['sunset (iso8601)'] - test_merged['sunrise (iso8601)']).dt.total_seconds()/3600

In [ ]:
test_merged.drop(['sunrise (iso8601)', 'sunset (iso8601)'], axis=1, inplace=True)

In [ ]:
test_merged['date'] = pd.to_datetime(test_merged['date'])

In [ ]:
test_merged['year'] = test_merged['date'].dt.year
test_merged['month'] = test_merged['date'].dt.month
test_merged['day'] = test_merged['date'].dt.day
test_merged.drop('date', axis=1, inplace=True)

In [ ]:
test_merged

,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),shortwave_radiation_sum (MJ/m²),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),et0_fao_evapotranspiration (mm)_x,...,soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),sunrise to sunset (hours),year,month,day
0,32.3,25.3,38.2,29.6,20.79,0.00,7.6,16.9,134.0,4.45,...,29.233333,28.600000,0.056500,0.077500,0.112500,0.234000,11.850000,2021,8,2
1,31.7,25.5,37.4,30.3,20.01,0.00,10.2,21.6,131.0,4.32,...,29.333333,28.600000,0.060958,0.076042,0.111583,0.233292,11.850000,2021,8,3
2,32.6,25.7,37.1,30.8,20.89,0.00,17.3,32.4,116.0,4.73,...,29.416667,28.687500,0.059542,0.074542,0.110917,0.233000,11.850000,2021,8,4
3,32.0,25.7,36.8,31.1,22.19,0.00,15.8,29.2,115.0,4.96,...,29.500000,28.700000,0.058500,0.073500,0.110375,0.232458,11.850000,2021,8,5
4,31.9,26.3,36.0,31.6,21.84,0.00,20.2,36.7,106.0,4.99,...,29.516667,28.700000,0.057750,0.072417,0.109458,0.232000,11.866667,2021,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4967,-6.9,-13.1,-13.5,-19.5,0.00,1.12,21.7,31.7,47.0,0.48,...,-9.575000,-8.058333,0.264000,0.205000,0.237000,0.252000,0.000000,2022,12,18
4968,-2.5,-6.8,-7.1,-13.4,0.00,1.40,21.6,31.0,64.0,0.51,...,-9.600000,-8.100000,0.264000,0.205000,0.237000,0.252000,0.000000,2022,12,19
4969,-4.6,-16.9,-9.0,-21.9,0.00,1.05,14.1,23.0,273.0,0.03,...,-9.600000,-8.154167,0.264000,0.205000,0.237000,0.252000,0.000000,2022,12,20
4970,-17.2,-21.2,-22.2,-26.5,0.00,0.00,10.8,19.1,293.0,0.00,...,-9.600000,-8.200000,0.264000,0.205000,0.237000,0.252000,0.000000,2022,12,21


In [ ]:
test_merged['city'] = test_merged['city'].map(city_dic)

In [ ]:
test_cat = test_merged[['city', 'year', 'month', 'day']]
test_num = test_merged.drop(['city', 'year', 'month', 'day'], axis=1)
test_num_scaled = pd.DataFrame(columns = test_num.columns, index=test_num.index, data=scaler.fit_transform(test_num))
test_final = pd.concat([test_cat, test_num_scaled], axis=1)
test_final

,city,year,month,day,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),shortwave_radiation_sum (MJ/m²),snowfall_sum (cm),...,vapor_pressure_deficit (kPa),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),sunrise to sunset (hours)
0,1,2021,8,2,0.741889,0.652541,0.700052,0.624841,0.680288,-0.160431,...,3.687667,1.013505,0.956664,0.846556,0.801378,-2.115468,-2.027396,-1.767948,-0.922402,0.142634
1,1,2021,8,3,0.692992,0.669609,0.650395,0.670583,0.561270,-0.160431,...,3.131747,0.968747,0.953696,0.855628,0.801378,-2.077699,-2.040158,-1.776169,-0.929118,0.142634
2,1,2021,8,4,0.766337,0.686676,0.631774,0.703256,0.695547,-0.160431,...,3.001121,0.977185,0.950729,0.863188,0.809472,-2.089700,-2.053284,-1.782148,-0.931883,0.142634
3,1,2021,8,5,0.717440,0.686676,0.613153,0.722860,0.893910,-0.160431,...,2.966186,0.959942,0.944423,0.870748,0.810628,-2.098525,-2.062400,-1.787006,-0.937019,0.142634
4,1,2021,8,6,0.709291,0.737879,0.563496,0.755533,0.840505,-0.160431,...,2.786955,0.943800,0.936633,0.872260,0.810628,-2.104879,-2.071880,-1.795227,-0.941365,0.147873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4967,10,2022,12,18,-2.452725,-2.624425,-2.509028,-2.583664,-2.492006,1.716150,...,-1.139416,-2.678658,-2.684591,-2.674082,-2.589730,-0.357587,-0.911663,-0.651349,-0.751738,-3.581667
4968,10,2022,12,19,-2.094146,-2.086798,-2.111772,-2.185052,-2.492006,2.185295,...,-1.013347,-2.662883,-2.676430,-2.676350,-2.593585,-0.357587,-0.911663,-0.651349,-0.751738,-3.581667
4969,10,2022,12,20,-2.265286,-2.948708,-2.229708,-2.740495,-2.492006,1.598863,...,-1.303458,-2.647841,-2.662334,-2.676350,-2.598596,-0.357587,-0.911663,-0.651349,-0.751738,-3.581667
4970,10,2022,12,21,-3.292127,-3.315660,-3.049047,-3.041089,-2.492006,-0.160431,...,-1.394592,-2.676457,-2.676430,-2.676350,-2.602835,-0.357587,-0.911663,-0.651349,-0.751738,-3.581667


In [ ]:
predictions = model.predict(test_merged)
result = pd.DataFrame({'rain_sum (mm)' : predictions})

In [ ]:
result.insert(0, 'id', result.index)

In [ ]:
result = result.set_index('id')

In [ ]:
result.to_csv('percobaan_kedua.csv')